In [1]:
import os
import pandas as pd
import re
import numpy as np
import collections
from operator import itemgetter
import pprint as pp
import re
from nltk.corpus import stopwords
import nltk.data
import time

#data from http://times.cs.uiuc.edu/~wang296/Data/
files = os.listdir('./Review_Texts')
#print(os.listdir('./Review_Texts'))
print(len(files))

1759


In [2]:
cats = ['Rooms', 'Date', 'Location', 'Service', 'Business service', 'Author', 'Check in / front desk', 'No. Helpful', 'Cleanliness', 'Content', 'Value', 'No. Reader', 'Overall']
data = pd.DataFrame(columns=cats)

In [3]:
def addFileToData(filename, data):
    intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']
    characterThreshold = 60
    with open(filename, 'r') as content_file:
        content = content_file.read()
        
    reviews = content.split("\n\n")
    for r in reviews:
        thisReview = pd.Series([None]*len(cats), cats)
        splt = r.split("\n")
        for s in splt:
            for c in cats:
                if "<"+c+">" in s:
                    value = s.replace('<'+c+'>', '')
                    if c in intColumns:
                        value = int(value)
                    if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
                        value = np.nan

                    if c == "Content":
                        value = value.lower()

                    thisReview[c] = value
                    
        if not thisReview["Content"] == None and len(thisReview["Content"]) > characterThreshold:
            #only add if theres content and its long enough
            data = data.append(thisReview, ignore_index=True)
    return data
        
data = addFileToData('./Review_Texts/hotel_100504.dat', data) 
data = addFileToData('./Review_Texts/hotel_72572.dat', data) 


In [4]:
print(data.shape)
data.head()

(199, 13)


,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4,Hilobb,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5,Chianti_girl24,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [9]:
#mean of int columns
data.mean(axis=0)

Rooms                    4.208589
Location                 4.296875
Service                  4.167702
Check in / front desk    4.273438
Cleanliness              4.518293
Value                    3.969136
Overall                  4.165829
dtype: float64

In [5]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
data["Content"][1]

"lovely hotel, unique decor, friendly front desk staff, central location it's only recently that i've started staying at boutique hotels, so i'm definitely not an expert on the subtleties that these types of establishments provide. but i've already stayed at a few kimptons and i must say that my experiences are mostly good. hotel monaco is no exception. the decor at the kimpton hotels is always unique - they have a wonderful way of pulling off colors, textures and patterns so one feels cool and trendy yet comfortable. the lobby and lounge were decorated really nicely in very rich tones. our room provided a nice view and had this modern, romantic, shabby-chic feel to it, for lack of a better description. all the staff was friendly and helpful without any attitude. sazerac restaurant served us delicious breakfast in a really great setting. seattle is a cute town in and of itself, so the hotel only enhanced our stay. the location was ideal! we walked everywhere and had no need for a taxi.

In [7]:
#creating the vocab of all the words

def lintWord(w):
    regex = re.compile(r'(,|\(|\)|!|:|$|\.)')
    w = re.sub(regex, '', w)
    return w


allWords = ""
for r in data["Content"]:
    #add word to big content string
    allWords += r + " "
    
#split the string at spaces, keep only unique
words = set(allWords.split(" "))


vocab = list(set([lintWord(w) for w in words if not w in stopwords.words("english")]))

#n eed to remove stopwords again because some of them may have had punctuation 
# at the end and didnt get caught the first time
vocab = [w for w in vocab if not w in stopwords.words("english") and len(w) > 2]

#vocab


C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:18: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:22: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [9]:
# Stemming of vocab
from nltk.stem.porter import *
stemmer=PorterStemmer()
stemmed_vocab = [stemmer.stem(i.decode('utf-8')) for i in vocab]

In [ ]:
#tokenizer to split sentences
from nltk.tokenize import RegexpTokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer2 = RegexpTokenizer(r'\w+')


LDA IMPLEMENTATION

In [25]:
# initial seeds from LARA paper
from gensim import corpora, models
seeds = {"Value" : ["value", "price", "quality","worth"],
         "Rooms" : ["room", "suite", "view", "bed"],
         "Location" : ["location", "traffic", "minute", "restaurant"],
         "Cleanliness" : ["clean", "dirty", "maintain", "smell"],
         "Check in / front desk": ["stuff", "check", "help", "reservation"],
         "Service" : ["service", "food", "breakfast", "buffet"],
         "Business service" : ["business", "center", "computer", "internet"]
        }


def aspectSegmentation(reviews, aspects, vocab=[], threshold=0, iterationLimit=3):
    #when we have the top chi-squared rated keywords, how many do we take
    keywordsToTake = 3
    
    #bootstrap iterations
    for i in range(0, iterationLimit):
        
        #print our current aspects
        print("begin bootstrapping iteration, aspect keywords: ")
        pp.pprint(aspects)
        print("\n\n\n")
        labeledSentences = []
        for r in reviews:
            
            #use the pickle tokenizer to split sentences
                 
            sentences = tokenizer.tokenize(r.decode('utf-8'))

                
            for s in sentences:
                
                theseAspects = collections.defaultdict(int)
                maxAspect = (0, "None")
                
                #for each aspect count how many times one of those aspect words appears
                for a in aspects:
                    for word in aspects[a]:
                        if " "+word+" " in s:
                            theseAspects[a] += 1
                
                #find the max occuring aspect for each sentence, take multiple if ties
                for a in theseAspects:
                    if theseAspects[a] > maxAspect[0]:
                        maxAspect = (theseAspects[a], a)
                    if theseAspects[a] == maxAspect[0] and a not in maxAspect:
                        #label it with multiple aspects
                        maxAspect = maxAspect + (a, )
                
                #add the sentence with labels
                labeledSentences.append((s, maxAspect[1:]))
            
        LDAForAspects = collections.defaultdict(list)
        
# obtain dictionary of tokenized sentences corresponding to each aspect
        for a in aspects:
            for s in labeledSentences:
                sentenceText=s[0]
                sentenceAspects=s[1]
                
                if a in sentenceAspects:
                    sentenceTextTokens=tokenizer2.tokenize(sentenceText)
                    sentenceTextTokensNS=[i for i in sentenceTextTokens if not i in stopwords.words("english") and not i in ['hotel']  ]
#                     sentenceTextTokensNS_stemmed = [stemmer.stem(i) for i in sentenceTextTokensNS]
                    LDAForAspects[a].extend(sentenceTextTokensNS)

#Implement LDA to obtain topics
        SentenceWords=[]
        dictionary={}
        corpus={}
        for a in aspects:
            SentenceWords.append(LDAForAspects[a])
            dictionary[a]=corpora.Dictionary([LDAForAspects[a]])
            corpus[a] = [dictionary[a].doc2bow([text]) for text in LDAForAspects[a]]
            ldamodel = models.ldamodel.LdaModel(corpus[a], num_topics=1, id2word = dictionary[a], passes=20)
            print a+":"
            print(ldamodel.print_topics(num_topics=1, num_words=5))
            X=ldamodel.get_topic_terms(0,3)
            i=0
            while i<len(X):
#                 print(list(dictionary[a].token2id.keys())[list(dictionary[a].token2id.values()).index(X[i][0])])
                temp=list(dictionary[a].token2id.keys())[list(dictionary[a].token2id.values()).index(X[i][0])]
                if temp not in aspects[a]:  
                    aspects[a].append(temp)
                i=i+1
            print aspects[a]
        
#         dictionary[a] = corpora.Dictionary(LDAForAspects)
#         corpus[a] = [dictionary[a].doc2bow(text) for text in LDAForAspects[a]]
#         #calculate chi squared measure for each word in vocab
#         """c1 is the number of times w occurs in sentences belonging
#             to aspect a_i, c2 is the number of times w occurs
#             in sentences not belonging to a_i, c3 is the number of sentences
#             of aspect a_i that do not contain w, c4 is the number
#             of sentences that neither belong to aspect a_i, nor contain
#             word w, and C is the total number of word occurrences"""
#         for w in vocab:
#             for a in aspects:
#                 c = 0
#                 c_1 = 0
#                 c_2 = 0 
#                 c_3 = 0
#                 c_4 = 0

#                 for s in labeledSentences:
#                     sentenceText = s[0]
#                     sentenceAspects = s[1]

#                     if " "+w+" " in sentenceText and a in sentenceAspects:
#                         c_1 += 1
#                     elif " "+w+" " in sentenceText and a not in sentenceAspects:
#                         c_2 += 1
#                     elif a in sentenceAspects and w not in sentenceText:
#                         c_3 += 1
#                     else:
#                         c_4 += 1

#                 numer = ((1.0 * c_1 * c_4 - 1.0 * c_2 * c_3)**2)
#                 denom = (1.0*(c_1 + c_3) * 1.0 * (c_2 + c_4) * 1.0 * (c_1 + c_2) * 1.0 * (c_3 + c_4))
#                 #unreasonable use of 1.0's here to be safe
#                 if denom != 0:
#                     csq = numer / denom
#                     chiSquaredForAspects[a].append((w, csq))


#         #have the chi squared aspects for each word in vocab, add top kewordsToTake for each aspect
#         for a in chiSquaredForAspects:
#             #make sure were not taking words we already have
#             noDupes = [tup for tup in chiSquaredForAspects[a] if tup[0] not in aspects[a]]
# #             chiSquaredForAspects[a] = sorted(noDupes, key=itemgetter(1), reverse=True)[0:keywordsToTake]
#             for t in chiSquaredForAspects[a]:
#                 if t[0] not in aspects[a] and t[0] != '':
#                     aspects[a].append(t[0])
            
            
        
            #split into sentences
        
        #loop through again
    return labeledSentences
    #return labeledSentences

start = time.time()
sentencesWLabels = aspectSegmentation(data["Content"], seeds, vocab)
end = time.time()

print("done, time taken:", end-start)

begin bootstrapping iteration, aspect keywords: 
{'Business service': ['business', 'center', 'computer', 'internet'],
 'Check in / front desk': ['stuff', 'check', 'help', 'reservation'],
 'Cleanliness': ['clean', 'dirty', 'maintain', 'smell'],
 'Location': ['location', 'traffic', 'minute', 'restaurant'],
 'Rooms': ['room', 'suite', 'view', 'bed'],
 'Service': ['service', 'food', 'breakfast', 'buffet'],
 'Value': ['value', 'price', 'quality', 'worth']}




Service:
[(0, u'0.042*service + 0.020*room + 0.018*breakfast + 0.013*great + 0.012*food')]
['service', 'food', 'breakfast', 'buffet', u'room']
Business service:
[(0, u'0.022*business + 0.014*internet + 0.008*access + 0.008*center + 0.008*room')]
['business', 'center', 'computer', 'internet', u'access']
Cleanliness:
[(0, u'0.047*clean + 0.024*room + 0.015*rooms + 0.012*staff + 0.011*nice')]
['clean', 'dirty', 'maintain', 'smell', u'room', u'rooms']
Check in / front desk:
[(0, u'0.020*check + 0.012*help + 0.010*desk + 0.010*reservation 

In [20]:
numToShow = 20
count = 0
for s in sentencesWLabels:
    if 'None' not in s[1]:
        print(s)
        print("\n")
        count += 1
    if count > numToShow:
        break

(u'what a great experience!', ('Location',))


(u'from the goldfish in the room (which my daughter loved) to the fact that the valet parking staff who put on my chains on for me it was fabulous.', ('Cleanliness', 'Check in / front desk', 'Rooms', 'Service'))


(u'the staff was attentive and went above and beyond to make our stay enjoyable.', ('Check in / front desk',))


(u"oh, and about the parking: the charge is about what you would pay at any garage or lot- and i bet they wouldn't help you out in the snow! ", ('Check in / front desk',))


(u"lovely hotel, unique decor, friendly front desk staff, central location it's only recently that i've started staying at boutique hotels, so i'm definitely not an expert on the subtleties that these types of establishments provide.", ('Check in / front desk',))


(u'hotel monaco is no exception.', ('Value', 'Business service'))


(u'our room provided a nice view and had this modern, romantic, shabby-chic feel to it, for lack of a better descripti